### Header Code

In [33]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
from datetime import datetime
import plotly.plotly as py
import plotly.graph_objs as go
import math
import time
from scipy.optimize import fmin
from scipy.optimize import minimize

## BAPM Functions

### Helper Functions

In [35]:
#generates the stock price vector for the BAPM function
def generate_stockPrices(u, d, row, col, matrix):
    if row > col:#if the row > column return matrix; value outside of the tree structure
        return matrix
    if ((row == 0) and (col == 0)):#return 1 for the first value
                matrix[row][col] = 1
    else:
        matrix[row][col] = u**(col - row) * d**(row)#return a combination of U & D for each value
    return matrix

#generates the state price vector for the BAPM function
def generate_statePrices(q, row, col, matrix):
    if col == row:
        matrix[row][col] = q
    if col - row == -1:
        matrix[col][row] = 1-q
    return matrix

#generates the final value of the option in the period; purely intrinsic value for both European & American
def generate_finalValue(call, strikePrice, value):
    if call:#calculate the call value
        value = value - strikePrice
    else:#calculate the put value
        value = strikePrice - value
    return np.clip(value, a_min=0, a_max=None)#clip all negative numbers; setting min to 0

#Generates the backwards recursive option price for both European and American Options
def generate_optionPrices(optionType, strikePrice, valueMatrix, priceMatrix, q, discount):
    exerciseLocations = []
    for j in range(valueMatrix.shape[1]-1, 0, -1):#loop backwards through columns first
        for i in range(1, valueMatrix.shape[0]):#loop downwards through rows; saves final spot of [0,0]
            if i > j:#Continue the loop if the row is greater than the column; Outside of the tree structure
                continue
            if not optionType:#european option pricing
                valueMatrix[i-1][j-1] = discount*(q*valueMatrix[i-1][j] + (1-q)*valueMatrix[i][j])
                
            else:#american option pricing
                continiousValue = discount*(q*valueMatrix[i-1][j] + (1-q)*valueMatrix[i][j])
                exerciseValue = priceMatrix[i-1][j-1] - strikePrice
                if exerciseValue > continiousValue:
                    exerciseLocations.append(tuple(i-1, j-1))
                
                valueMatrix[i-1][j-1] = max(exerciseValue, continiousValue)
    print('\nExercise Locations: ', exerciseLocations)
                
    return valueMatrix

def generate_plots(prices, values):
    trace = go.Scatter(
    x = values,
    mode = 'markers'
    )
    
    data = [trace]

    return py.iplot(data, filename='option-values')

### Main Function

In [40]:
def bapm(curPrice, strikePrice, volatility, monthTerm, riskFree, numPeriods):
    
    #Section: Asks user for Call/Put and European/American
    print('0: Put\n1: Call')
    call = None
    while call not in [0, 1]:
        call = int(input('Select a 0 or 1: '))
    print('\n0: European\n1: American')
    optionType = None
    while optionType not in [0, 1]:
        optionType = int(input('Select a 0 or 1 for option type: '))
    
    #Section: setting variables up
    start = time.clock()
    monthTerm = monthTerm/12
    timeStep = monthTerm/numPeriods
    u = np.exp(volatility*(np.sqrt(timeStep)))
    d = u**-1
    q = (np.exp(riskFree*timeStep) - d)/(u-d)
    discount = np.exp(-riskFree*timeStep)
    priceMatrix = np.zeros((numPeriods+1, numPeriods+1))
    statePrices = np.zeros((numPeriods+1, numPeriods+1))
    valueMatrix = np.zeros((numPeriods+1, numPeriods+1))
    
    #Section: Loops through state prices and generates Stock Prices matrix
    for i in range(0, numPeriods + 1):#row loop
        for j in range(0, numPeriods + 1):#col loop
            #I chose to loop through the cols first and then rows
            statePrices = generate_statePrices(q=q, row=i, col=j, matrix=statePrices)
            priceMatrix = generate_stockPrices(u=u, d=d, row=i, col=j, matrix=priceMatrix)
    
    #Section: Function Implementation and further calculations
    priceMatrix = curPrice*priceMatrix#Adds the price to price matrix
    valueMatrix[:, -1] = generate_finalValue(call=call, strikePrice=strikePrice, value=priceMatrix[:, -1])
    optionPrices = generate_optionPrices(optionType=optionType, strikePrice=strikePrice,
                                         valueMatrix=valueMatrix, priceMatrix=priceMatrix,
                                         q=q, discount=discount)
    
    #############################################################################################################
    #TESTING FUNCTIONS
    #print('Price Matrix:\n', priceMatrix, '\n')
    #print('Value Matrix:\n', valueMatrix)
    #print('State Prices Matrix:\n', statePrices, '\n')
    #print('Option Value --  State Prices Method --:\n', np.dot((discount*statePrices)**numPeriods, valueMatrix[:, -1]), '\n')
    print('\nOption Values -- backwards loop --:\n', optionPrices)
    #generate_plots(prices=None, values=optionPrices)
    #########################################################################################################
    end = time.clock()
    #print("Time Elapsed: ", end-start)
    
    return optionPrices[0][0]

### Calculate Time = 0 Options Price

In [ ]:
bapm(strikePrice=40, curPrice=40, volatility=.3, monthTerm=6, riskFree=.04, numPeriods=10)

## Backing Out Volatility; Double checking answers

In [26]:
def findVol(vol):
    calculatedPrice = bapm(strikePrice=40, curPrice=40, volatility=vol, monthTerm=6, riskFree=.04, numPeriods=2)
    realPrice = 3.37398
    
    return calculatedPrice - realPrice

In [27]:
#parameters = [.3]
#res = minimize(fun=findVol, x0=parameters)
#print('Parameters: ', res.x)

Option Value --  State Prices Method --:
 [3.37391914 0.         0.        ] 

Option Values -- backwards loop --:
 [[ 3.37391914  6.87137636 13.9943523 ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]
Option Value --  State Prices Method --:
 [3.37391928 0.         0.        ] 

Option Values -- backwards loop --:
 [[ 3.37391928  6.87137671 13.99435311]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]
Option Value --  State Prices Method --:
 [3.37391914 0.         0.        ] 

Option Values -- backwards loop --:
 [[ 3.37391914  6.87137636 13.9943523 ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]
Option Value --  State Prices Method --:
 [ 0.          7.35828629 13.35566098] 

Option Values -- backwards loop --:
 [[ 7.35828629  0.          0.        ]
 [ 0.         17.44523282  0.        ]
 [ 0.          0.         41.35965035]]
Option Value --  State Prices Method --:
 [ 0.          7.358